In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras

/Users/randyngo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# df2009 = pd.read_parquet('ss2024.parquet')
# df2014 = pd.read_parquet('ss2024.parquet')
# df2019 = pd.read_parquet('ss2024.parquet')
# df2024 = pd.read_parquet('ss2024.parquet')
df2009 = pd.read_parquet('subset2009.parquet')
df2014 = pd.read_parquet('subset2014.parquet')
df2019 = pd.read_parquet('subset2019.parquet')
df2024 = pd.read_parquet('subset2024.parquet')

In [3]:
# df2024.shape

In [4]:
zones = gpd.read_file('taxi_zones/taxi_zones.shp')

In [5]:
# zones['borough'].unique()

In [6]:
boroughs = zones[['LocationID', 'borough']]
boroughs.shape

(263, 2)

In [7]:
df2009.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,Start_Lon,Start_Lat,End_Lon,End_Lat,payment_type,fare_amount,extra,tip_amount,tolls_amount,hour,day_of_week,month,year
0,CMT,2009-01-01 00:00:37,2009-01-01 00:07:31,1,1.4,-73.993538,40.727442,-74.006315,40.739391,1,7.0,0.0,1.00,0.0,0,3,1,2009
1,CMT,2009-01-01 00:05:40,2009-01-01 00:10:04,1,0.4,-74.000174,40.730405,-73.999066,40.734198,1,4.6,0.0,2.00,0.0,0,3,1,2009
2,CMT,2009-01-01 00:09:23,2009-01-01 00:15:32,2,1.1,-73.958610,40.784296,-73.942775,40.786082,1,6.2,0.0,1.55,0.0,0,3,1,2009
3,CMT,2009-01-01 00:09:43,2009-01-01 00:26:57,3,6.0,-73.955187,40.768987,-74.005795,40.733189,1,17.4,0.0,2.61,0.0,0,3,1,2009
4,CMT,2009-01-01 00:10:57,2009-01-01 00:32:19,4,4.7,-73.984541,40.771447,-73.978839,40.730922,1,15.4,0.0,2.00,0.0,0,3,1,2009


In [8]:
# new09 = pd.merge(df2009, boroughs, how='left')
# new09 = new09.dropna()
# new09 = new09[new09["payment_type"] == 1]
new09 = df2009.drop(['VendorID'],axis=1)
new09 = new09.drop(['Start_Lon'],axis=1)
new09 = new09.drop(['Start_Lat'],axis=1)
new09 = new09.drop(['End_Lon'],axis=1)
new09 = new09.drop(['End_Lat'],axis=1)

new14 = pd.merge(df2014, boroughs, how='left', left_on='PULocationID', right_on='LocationID')
new14 = new14.dropna()
new14 = new14[new14["payment_type"] == 1]

new19 = pd.merge(df2019, boroughs, how='left', left_on='PULocationID', right_on='LocationID')
new19 = new19.dropna()
new19 = new19[new19["payment_type"] == 1]

new24 = pd.merge(df2024, boroughs, how='left', left_on='PULocationID', right_on='LocationID')
new24 = new24.dropna()
new24 = new24[new24["payment_type"] == 1]

In [9]:
# new.head()
# new['borough'].unique()
# new[new['borough'].isnull()]

In [10]:
X = new09.drop(['tip_amount'],axis=1)
# onehot = pd.get_dummies(X['borough'])
# X = X.drop(['borough'],axis=1)
# X = X.join(onehot)
X = X.drop(['tpep_pickup_datetime'],axis=1)
X = X.drop(['tpep_dropoff_datetime'],axis=1)

y = new09['tip_amount']

X_train09, X_test09, y_train09, y_test09 = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
X = new14.drop(['tip_amount'],axis=1)
onehot = pd.get_dummies(X['borough'])
X = X.drop(['borough'],axis=1)
X = X.join(onehot)
X = X.drop(['tpep_pickup_datetime'],axis=1)
X = X.drop(['tpep_dropoff_datetime'],axis=1)

y = new14['tip_amount']

X_train14, X_test14, y_train14, y_test14 = train_test_split(X, y, test_size=0.1, random_state=42)

In [12]:
X = new19.drop(['tip_amount'],axis=1)
onehot = pd.get_dummies(X['borough'])
X = X.drop(['borough'],axis=1)
X = X.join(onehot)
X = X.drop(['tpep_pickup_datetime'],axis=1)
X = X.drop(['tpep_dropoff_datetime'],axis=1)

y = new19['tip_amount']

X_train19, X_test19, y_train19, y_test19 = train_test_split(X, y, test_size=0.1, random_state=42)

In [13]:
# X = df2024.drop(['tip_amount'],axis=1)
X = new24.drop(['tip_amount'],axis=1)
onehot = pd.get_dummies(X['borough'])
X = X.drop(['borough'],axis=1)
X = X.join(onehot)
X = X.drop(['tpep_pickup_datetime'],axis=1)
X = X.drop(['tpep_dropoff_datetime'],axis=1)

y = new24['tip_amount']
# y = new24['tip_amount'] / new24['fare_amount']

X_train24, X_test24, y_train24, y_test24 = train_test_split(X, y, test_size=0.1, random_state=42)

In [14]:
# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)

# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f"Random Forest Model Performance:")
# print(f"RMSE: {rmse:.2f}")
# print(f"R² Score: {r2:.4f}")

In [15]:
# rf.feature_importances_

In [16]:
# myClassifiers = pd.Series(rf.feature_importances_, index=X_train.columns)
# myClassifiers

# Neural Network

In [17]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

In [19]:
model = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # One output for regression
])

# model = Sequential([
#     Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
#     Dense(64, activation='relu'),
#     Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     Dense(1)  # One output for regression
# ])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['root_mean_squared_error'])  # Use MAE or RMSE for regression

# Train the model
history = model.fit(X_train09, y_train09, epochs=20, batch_size=64, validation_split=0.1)

# Evaluate the model
loss, rmse = model.evaluate(X_test09, y_test09)
print(f"Test RMSE: {rmse:.4f}")


Epoch 1/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 14s 620us/step - loss: 127.3245 - root_mean_squared_error: 7.5752 - val_loss: 2.2589 - val_root_mean_squared_error: 1.5029
Epoch 2/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 13s 609us/step - loss: 2.5440 - root_mean_squared_error: 1.5929 - val_loss: 2.3619 - val_root_mean_squared_error: 1.5368
Epoch 3/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 13s 616us/step - loss: 2.4020 - root_mean_squared_error: 1.5495 - val_loss: 2.1535 - val_root_mean_squared_error: 1.4675
Epoch 4/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 13s 598us/step - loss: 2.2930 - root_mean_squared_error: 1.5142 - val_loss: 2.1468 - val_root_mean_squared_error: 1.4652
Epoch 5/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 13s 606us/step - loss: 2.3526 - root_mean_squared_error: 1.5332 - val_loss: 2.1553 - val_root_mean_squared_error: 1.4681
Epoch 6/20
21630/21630 ━━━━━━━━━━━━━━━━━━━━ 13s 594us/step - loss: 2.2371 - root_mean_squared_error: 1.4954 - val_loss: 2.1495 - val_root_mean_squared_error: 1.4661
Epoch 7/

In [ ]:
y_pred09 = model.predict(X_test09)

r2 = r2_score(y_test09, y_pred09)
print(f"R² Score: {r2:.4f}")

In [ ]:
# history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.1)

# Plot loss over time
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Time')
plt.legend()

# RMSE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Model RMSE Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # One output for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['root_mean_squared_error'])  # Use MAE or RMSE for regression

# Train the model
history = model.fit(X_train14, y_train14, epochs=20, batch_size=64, validation_split=0.1)

# Evaluate the model
loss, rmse = model.evaluate(X_test14, y_test14)
print(f"Test RMSE: {rmse:.4f}")


In [ ]:
y_pred14 = model.predict(X_test14)

r2 = r2_score(y_test14, y_pred14)
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot loss over time
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Time')
plt.legend()

# RMSE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Model RMSE Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # One output for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['root_mean_squared_error'])  # Use MAE or RMSE for regression

# Train the model
history = model.fit(X_train19, y_train19, epochs=20, batch_size=64, validation_split=0.1)

# Evaluate the model
loss, rmse = model.evaluate(X_test19, y_test19)
print(f"Test RMSE: {rmse:.4f}")


In [ ]:
y_pred19 = model.predict(X_test19)

r2 = r2_score(y_test19, y_pred19)
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot loss over time
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Time')
plt.legend()

# RMSE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Model RMSE Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model = Sequential([
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # One output for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['root_mean_squared_error'])  # Use MAE or RMSE for regression

# Train the model
history = model.fit(X_train24, y_train24, epochs=20, batch_size=64, validation_split=0.1)

# Evaluate the model
loss, rmse = model.evaluate(X_test24, y_test24)
print(f"Test RMSE: {rmse:.4f}")


In [ ]:
y_pred24 = model.predict(X_test24)

r2 = r2_score(y_test24, y_pred24)
print(f"R² Score: {r2:.4f}")

In [ ]:
# Plot loss over time
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Over Time')
plt.legend()

# RMSE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Training RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Validation RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.title('Model RMSE Over Time')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# # Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
# model2 = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
#     Dense(64, activation='relu'),
#     Dense(1)  # One output for regression
# ])

# # Compile the model
# model2.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Use MAE or RMSE for regression

# # Train the model
# model2.fit(X_train_scaled, y_train, epochs=20, batch_size=64, validation_split=0.1)

# # Evaluate the model
# loss, mae = model2.evaluate(X_test_scaled, y_test)
# print(f"Test MAE: {mae:.4f}")

In [ ]:
# y_pred2 = model2.predict(X_test_scaled)

# r2 = r2_score(y_test, y_pred)
# print(f"R² Score: {r2:.4f}")

In [ ]:
# hist = pd.DataFrame(y_pred2.history)

# ## Plot our history:
# fig, ax1 = plt.subplots(2,1, figsize=(9,6))

# hist[['loss', 'val_loss']].plot(ax=ax1[0])
# hist[['accuracy', 'val_accuracy']].plot(ax=ax1[1])